In [1]:
import pandas as pd

In [41]:
filepath_or_buffer = '../data/articles.csv'

In [42]:
#import a data frame with 4 columns: orders, titles, descriptions, and articles
df = pd.read_csv(filepath_or_buffer)
num_of_articles=df.shape[0]
print "number of articles: ",num_of_articles

number of articles:  244


In [43]:
header_df = list(df)
print header_df

['Unnamed: 0', 'title', 'description', 'feed_article']


In [44]:
feed_article = df['feed_article']

In [45]:
from bs4 import BeautifulSoup
feed_article_clean = []
for text in feed_article:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    feed_article_clean.append(text)

In [46]:
print feed_article_clean

[u'Mount Sinjar: Islamic State siege broken, say Kurds\nKurdish security forces have been trying to stem the IS advance across Iraq\'s northern Nineveh province\nWhat the Kassig video tells us\nKurdish forces in northern Iraq are claiming their biggest victory yet against Islamic State (IS) militants.\nThey say they have broken the IS siege of Mount Sinjar, where thousands of Yazidis and other displaced Iraqis have been trapped since August.\nIS controls a swathe of Iraq and Syria, where it has declared a caliphate.\nMeanwhile, the Pentagon\'s top officer says US air strikes have killed several high-ranking military leaders of IS in Iraq.\nThe Kurdish offensive against IS forces besieging Mount Sinjar began early on Wednesday with the most intensive round of air strikes yet by US and coalition forces - 45 in all.\nKurdish security forces have been trying to stem the IS advance across Iraq\'s northern Nineveh province\nKurdish forces opened a wide corridor, allowing those trapped by IS 

In [47]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
import re
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [48]:
import nltk
token_stems = []
for i in range(num_of_articles):
    print i
    token_stems.append(tokenize_and_stem(feed_article_clean[i]))
#print token_stems

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243


In [49]:
import csv
csvfile='tokenized_stems.csv'
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in token_stems:
        print val
        writer.writerow([val])    

[u'mount', u'sinjar', u'islam', u'state', u'sieg', u'broken', u'say', u'kurd', u'kurdish', u'secur', u'forc', u'have', u'been', u'tri', u'to', u'stem', u'the', u'is', u'advanc', u'across', u'iraq', u"'s", u'northern', u'nineveh', u'provinc', u'what', u'the', u'kassig', u'video', u'tell', u'us', u'kurdish', u'forc', u'in', u'northern', u'iraq', u'are', u'claim', u'their', u'biggest', u'victori', u'yet', u'against', u'islam', u'state', u'is', u'milit', u'they', u'say', u'they', u'have', u'broken', u'the', u'is', u'sieg', u'of', u'mount', u'sinjar', u'where', u'thousand', u'of', u'yazidi', u'and', u'other', u'displac', u'iraqi', u'have', u'been', u'trap', u'sinc', u'august', u'is', u'control', u'a', u'swath', u'of', u'iraq', u'and', u'syria', u'where', u'it', u'has', u'declar', u'a', u'caliph', u'meanwhil', u'the', u'pentagon', u"'s", u'top', u'offic', u'say', u'us', u'air', u'strike', u'have', u'kill', u'sever', u'high-rank', u'militari', u'leader', u'of', u'is', u'in', u'iraq', u'the', 

In [12]:
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(feed_article_clean)
print(tfidf_matrix.shape)
num_articles = tfidf_matrix.shape[0]
print(num_articles)
num_terms = tfidf_matrix.shape[1]
print(num_terms)

CPU times: user 7.47 s, sys: 127 ms, total: 7.6 s
Wall time: 10.8 s
(244, 5492)
244
5492


In [10]:
tfidf_mat = tfidf_matrix.toarray()
print tfidf_mat

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.03855564
   0.03855564]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [16]:
import numpy as np

tfidf_df = pd.DataFrame(data=tfidf_mat.astype(float))
tfidf_df.to_csv('tfidf_mat.csv', sep=' ', header=False, float_format='%.5f', index=False)